creating custom Pyfunc models

In [5]:
import mlflow
import pathlib as Path

In [10]:
project_root = Path.cwd()  # or however you define root
mlruns_path = project_root / "mlruns"

mlflow.set_tracking_uri(mlruns_path.as_uri())


In [11]:
import pandas as pd

In [12]:
mlflow.set_experiment(experiment_name = "custom models with pyfunc")

2025/09/12 13:29:30 INFO mlflow.tracking.fluent: Experiment with name 'custom models with pyfunc' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/Mlflow/mlruns/258863656514453625', creation_time=1757662168680, experiment_id='258863656514453625', last_update_time=1757662168680, lifecycle_stage='active', name='custom models with pyfunc', tags={}>

In [13]:
def predict(model_input):
    return model_input.apply(lambda x:x *2)

with mlflow.start_run(run_name = "function_model"):
    mlflow.pyfunc.log_model("model", python_model = predict, pip_requirements = ['pandas'])
    run_id = mlflow.active_run().info.run_id

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\__init__.py:3146: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/09/12 13:39:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [15]:
model = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")
x_new = pd.Series([1,2,3,4,5])

prediction = model.predict(x_new)
print(prediction)

0     2
1     4
2     6
3     8
4    10
dtype: int64


Class-based Model

In [20]:
class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input, params = None):
        return [x*2 for x in model_input]


with mlflow.start_run():
    mlflow.pyfunc.log_model("model", python_model = MyModel(), pip_requirements = ["pandas"])
    run_id = mlflow.active_run().info.run_id

model = mlflow.pyfunc.load_model(f"runs:/{run_id}/model")
x_new = pd.Series([1,2,3])

print(f"prediction: {model.predict(x_new)}")

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/09/12 13:57:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


prediction: [2, 4, 6]


In [23]:
class CustomModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        self._preprocessed(model_input)
        return model_input.apply(lambda x: x*2)

    def _preprocessed(self, model_input):
        print("processing input..................")
        print("Input Length: ", len(model_input))

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [24]:
with mlflow.start_run(run_name = "class_model") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model("model", python_model = CustomModel(), pip_requirements = ["pandas"])

2025/09/12 14:10:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


992930d0737140428ed1b91a622e5204


In [26]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run.info.run_id}/model")

loaded_model.predict(x_new)

processing input..................
Input Length:  3


0    2
1    4
2    6
dtype: int64

Obtaining the original Class

In [28]:
class CustomModel(mlflow.pyfunc.PythonModel):
    def __init__(self):
        pass

    def predict(self, context, model_input):
        self.__preprocess(model_input)
        return model_input.apply(lambda x:x*2)
    
    def __preprocess(self, model_input):
        print("processing......")
        print("input length", len(model_input))

    def additional_method(self):
        print("additional method")

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [30]:
with mlflow.start_run(run_name = "class_model_with_additional_methods") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model("model", python_model = CustomModel(), pip_requirements = ["Pandas"])

2025/09/12 14:18:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


f4698fb066f04e19be5ea5869299360c


In [32]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run.info.run_id}/model")
prediction = loaded_model.predict(x_new)
print(prediction)

processing......
input length 3
0    2
1    4
2    6
dtype: int64


In [33]:
type(loaded_model)

mlflow.pyfunc.PyFuncModel

In [34]:
original_class = loaded_model.unwrap_python_model()

In [35]:
original_class.additional_method()

additional method


In [37]:
type(original_class)

__main__.CustomModel